In [ ]:
### SQL 연습문제 11-4

# dataflix_covid_dataset 데이터 세트, world_covid 테이블에서

# 2020년 6월 1일 데이터 중에서 사망자수가 높은 지역(combined_key)과 순위를 조회하세요.

# 이때, 동일 값인 경우 동일 순위가 부여되고, 다음 순위는 건너뛰지 않고 순차 번호로 부여합니다.

# - 조회 항목 : 발생날짜(refresh_date), 지역명(combined_key), deaths(사망자 수), 사망자 수 순위(deaths_rank)
# - 정렬 조건 : 사망자 수(deaths)가 높은 순

SELECT
  refresh_date,
  combined_key,
  deaths,
  DENSE_RANK() OVER ( ORDER BY deaths DESC ) AS deaths_rank
FROM `dataflix_covid_dataset.world_covid`
WHERE refresh_date = '2020-06-01'
ORDER BY deaths DESC;

In [ ]:
# 예제 타이핑
SELECT
  id,
  first_name,
  last_name,
  lag(id) over(order by id) as id_prev,
  lag(first_name) over(order by id) as first_name_prev,
  lead(id) over(order by id) as id_next,
  lead(first_name) over(order by id) as first_name_next,
FROM `thelook_ecommerce.users`
WHERE id in (1,2,3,4,5)
order by id;

In [ ]:
### SQL 연습문제 11-3

# dataflix_covid_dataset 데이터 세트, world_covid 테이블에서
# 전날 대비 확진자 수(confirmed_prior_day), 전날 대비 사망자 수(deaths_prior_day)를 조회하시오.

# - 조회 항목 : 국가명(country_name), 시도명(state_name), 발생날짜(refresh_date),
#               confirmed(확진자 수), 전날 대비 확진자 수(diff_confirmed_prior_day),
#               deaths(사망자 수), 전날 대비 사망자 수(diff_deaths_prior_day)
# - 정렬 조건 : 국가명(country_name)오름차순,
#               시도명(state_name) 순으로 오름차순, 발생날짜(refresh_date) 오름차순

SELECT
  country_name,
  state_name,
  refresh_date,
  confirmed,
  confirmed - LAG(confirmed) OVER (
    PARTITION BY country_name, state_name
    ORDER BY refresh_date ) AS diff_confirmed_prior_day,
  deaths,
  deaths - LAG(deaths) OVER (
    PARTITION BY country_name, state_name
    ORDER BY refresh_date ) AS diff_deaths_prior_day
FROM `dataflix_covid_dataset.world_covid`
ORDER BY country_name, state_name, refresh_date;

In [ ]:
### SQL 연습문제 11-5

# dataflix_covid_dataset 데이터 세트, world_covid 테이블에서

# 2022년 9월 3일의 지역별 사망자 순위에 따른 누적 분포 비율을 조회하시오.

# - 조회 항목 : 지역명(combined_key), 확진자 수(confirmed),  사망자수(deaths),
#               사망자 순위에 따른 누적 분포 비율(cume_dist)
# - 정렬 조건 : 사망자 수(deaths)가 높은 순

SELECT
  combined_key,
  confirmed,
  deaths,
  CUME_DIST() OVER (ORDER BY deaths DESC) AS cume_dist
FROM `dataflix_covid_dataset.world_covid`
WHERE refresh_date = '2022-09-03'
ORDER BY deaths DESC

In [ ]:
### SQL 연습문제 11-6

# dataflix_covid_dataset 데이터 세트, world_covid 테이블에서

# 2022년 9월 3일의 확진자 수를 5개의 영역으로 구분하여 조회하시오.

# - 조회 항목 : 지역명(combined_key), 확진자 수(confirmed), 영역(1,2,3,4,5 중 하나) - 확진자수 높은순 기준
# - 정렬 조건 : 확진자 수(confirmed)가 높은 순

SELECT
  combined_key,
  confirmed,
  NTILE(5) OVER ( ORDER BY confirmed desc),
FROM `dataflix_covid_dataset.world_covid`
WHERE refresh_date = '2022-09-03'
ORDER BY confirmed DESC

In [ ]:
SELECT
  id,
  order_id,
  user_id,
  product_id,
  ROUND(sale_price,2) AS sale_price,
  ROUND(SUM(sale_price)
    OVER(
      PARTITION BY user_id
      ORDER BY created_at
      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
    ,2) AS cumulative_sum_amout
FROM `thelook_ecommerce.order_items`
ORDER BY user_id, created_at

In [ ]:
## 2.1 2022년 전체 주문건, 총 매출, 평균 매출

# order_items 테이블에서 2022년도의 전체 주문건수, 총 매출, 평균 매출을 조회해보세요.

# **조회 항목**

# - 전체 주문 건수 (total_order_count)
# - 판매 금액 합계_(total_sale_price)
# - 평균 판매 금액 (avg_sale_price)

SELECT
  COUNT(DISTINCT order_id) AS total_order_count,
  ROUND(SUM(sale_price), 2) AS total_sale_price,
  ROUND(AVG(sale_price), 2) AS avg_sale_price,
FROM `thelook_ecommerce.order_items`
WHERE EXTRACT(YEAR FROM created_at) = 2022

In [ ]:
### **2.2.1 2022년 월별 주문건, 총 매출, 평균 매출**

# order_items 테이블에서 2022년도의 월별 주문건수, 총 판매금액, 평균 판매금액을 조회해보세요.

# **조회 항목**

# - 월(month)
# - 주문 건수 (total_order_count)
# - 판매 금액 합계 (total_sale_price)
# - 평균 판매금액 (avg_sale_price)

# **정렬 순서**

# - 월(month) 오름차순

SELECT
  EXTRACT(MONTH FROM created_at) AS month,
  COUNT(DISTINCT order_id) AS total_order_count,
  ROUND(SUM(sale_price), 2) AS total_sale_price,
  ROUND(AVG(sale_price), 2) AS avg_sale_price,
FROM `thelook_ecommerce.order_items`
WHERE EXTRACT(YEAR FROM created_at) = 2022
GROUP BY month
ORDER BY month

In [ ]:
### **2.2.2 모든 연도의 월별 주문건, 총 매출, 평균 매출**

# order_items 테이블에서 2022연도까지의 모든연도의 주문문건수, 총 판매금액, 평균 판매금액을 조회해보세요.

# **조회 항목**

# - 월(month)
# - 주문 건수 (total_order_count)
# - 판매 금액 합계 (total_sale_price)
# - 평균 판매금액 (avg_sale_price)

# **정렬 순서**

# - 월(month) 오름차순

SELECT
  EXTRACT(MONTH FROM created_at) AS month,
  COUNT(DISTINCT order_id) AS total_order_count,
  ROUND(SUM(sale_price), 2) AS total_sale_price,
  ROUND(AVG(sale_price), 2) AS avg_sale_price,
FROM `thelook_ecommerce.order_items`
WHERE EXTRACT(YEAR FROM created_at) <= 2022
GROUP BY month
ORDER BY month

In [ ]:
### **2.2.3 2022년도와** 2022년도 이하 모든연도의 **월별 주문건수, 판매금액합계, 평균판매금액**

# order_items 테이블에서 월별 주문건수, 판매금액합계, 평균 판매금액을 조회해보세요.

# **조회 항목**

# - 월(month)
# - 2022년도의 월 주문건수 (order_count_2022)
# - 모든 연도의 월 주문건수 (order_count_total)
# - 2022년도의 월 판매금액 합계 (sum_sale_price_2022)
# - 모든 연도의 월 판매금액 합계 (sum_sale_price_total)
# - 2022년도의 월 평균판매금액 (avg_sale_price_2022)
# - 모든 연도의 월 평균 판매금액 (avg_sale_price_total)

# **정렬 순서**

# - 월(month) 오름차순

WITH sale_2022 AS
(
  SELECT
    EXTRACT(MONTH FROM created_at) AS month,
    COUNT(DISTINCT order_id) AS total_order_count,
    ROUND(SUM(sale_price), 2) AS total_sale_price,
    ROUND(AVG(sale_price), 2) AS avg_sale_price,
  FROM `thelook_ecommerce.order_items`
  WHERE EXTRACT(YEAR FROM created_at) = 2022
  GROUP BY month
  ORDER BY month
),
sale_total AS (
  SELECT
    EXTRACT(MONTH FROM created_at) AS month,
    COUNT(DISTINCT order_id) AS total_order_count,
    ROUND(SUM(sale_price), 2) AS total_sale_price,
    ROUND(AVG(sale_price), 2) AS avg_sale_price,
  FROM `thelook_ecommerce.order_items`
  WHERE EXTRACT(YEAR FROM created_at) <= 2022
  GROUP BY month
  ORDER BY month
  )
SELECT
  t1.month,
  t1.total_order_count AS order_count_2022,
  t2.total_order_count AS order_count_total,
  t1.total_sale_price AS sum_sale_price_2022,
  t2.total_sale_price AS sum_sale_price_total,
  t1.avg_sale_price AS avg_sale_price_2022,
  t2.avg_sale_price AS avg_sale_price_total
FROM sale_2022 t1
JOIN sale_total t2 ON t1.month = t2.month
ORDER BY month
